In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from mpl_toolkits.mplot3d import Axes3D
import pylab
import scipy.stats as stats
from scipy.stats import shapiro
import warnings
import os
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
df.head()

In [ ]:
df.rename(columns={'Annual Income (k$)':'AnnualIncome',
                   'Spending Score (1-100)':'SpendingScore'},inplace=True)

In [ ]:
df.describe()

In [ ]:
print(f'CustomerID :{df.CustomerID.nunique()}')
print(f'Gender :{df.Gender.nunique()}')
print(f'Age :{df.Age.nunique()}')
print(df.shape[0])

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df2 = df.copy()
cat_cols = ['Gender']
con_cols = ['Age','AnnualIncome','SpendingScore']

In [ ]:
# compute the corr matrix

corr = df.corr()

# generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr,dtype=bool))

# set up the matplotlib figure
f, ax = plt.subplots(figsize=(11,9))

# generate a custom diverging colormap
cmap = sns.diverging_palette(230,20,as_cmap=True)

#draw the heatpmap with the mask and correct aspect ratio
sns.heatmap(corr,mask=mask,cmap=cmap,vmax=.3,center=0,square=True,linewidths=.5,cbar_kws={'shrink':.5})

In [ ]:
sns.countplot(data=df,x='Gender');

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = sns.displot(data=df,x='Age',kde=True,color='blue');

In [ ]:
fig = plt.figure(figsize=(8,8))
sns.displot(data=df,x='AnnualIncome',kde=True,color='green');

In [ ]:
fig = plt.figure(figsize=(8,8))
sns.displot(data=df,x='SpendingScore',kde=True,color='red');

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.pairplot(df.drop('CustomerID', axis=1), hue='Gender', aspect=1.5);

In [ ]:
sns.relplot(
    data=df, x="SpendingScore", y="Age",hue="Gender",
    palette=["b", "r"], sizes=(10, 100)
);

In [ ]:
ax = sns.relplot(
    data=df, x="AnnualIncome", y="SpendingScore", col="Gender",
    hue="Gender", kind="line");

In [ ]:
sns.scatterplot(data=df, y="SpendingScore", x="AnnualIncome", hue="Gender", style="Gender");

In [ ]:
ax = sns.stripplot(x=df["SpendingScore"])

In [ ]:
plt.figure(figsize=(19,5))
sns.set_theme(style='darkgrid')
plt.subplot(1,3,1)
sns.violinplot(x='SpendingScore',y='Gender',data=df)
plt.subplot(1,3,2)
sns.violinplot(x='AnnualIncome',y='Gender',data=df)
plt.subplot(1,3,3)
sns.violinplot(x='Age',y='Gender',data=df)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(3,3,1)
sns.boxplot(y='Age',data=df);
ax2 = fig.add_subplot(3,3,2)
sns.boxplot(y='AnnualIncome',data=df);
ax3 = fig.add_subplot(3,3,3)
sns.boxplot(y='SpendingScore',data=df);

In [ ]:
shapiro_test = shapiro(df['SpendingScore'])

print(f'W: {shapiro_test[0]}')
print(f'p-Value: {shapiro_test[1]}')

# H0: Distribution is Normal!
# H1: Distribution is not Normal!

# Fail to Reject H0
# Reject H0
    
if shapiro_test[1] > 0.05:
  print('Fail to Reject H0 and Distribution is Normal!')
else:
  print('Reject H0 and Distribution is not Normal! ') 

In [ ]:
stats.probplot(df['SpendingScore'],plot=pylab,dist='norm')
pylab.show()

In [ ]:
shapiro_test = shapiro(df['AnnualIncome'])

print(f'W: {shapiro_test[0]}')
print(f'p-Value: {shapiro_test[1]}')

# H0: Distribution is Normal!
# H1: Distribution is not Normal!

# Fail to Reject H0
# Reject H0
    
if shapiro_test[1] > 0.05:
  print('Fail to Reject H0 and Distribution is Normal!')
else:
  print('Reject H0 and Distribution is not Normal! ') 

In [ ]:
stats.probplot(df['AnnualIncome'],plot=pylab,dist='norm')
pylab.show()

In [ ]:
shapiro_test = shapiro(df['Age'])

print(f'W: {shapiro_test[0]}')
print(f'p-Value: {shapiro_test[1]}')

# H0: Distribution is Normal!
# H1: Distribution is not Normal!

# Fail to Reject H0
# Reject H0
    
if shapiro_test[1] > 0.05:
  print('Fail to Reject H0 and Distribution is Normal!')
else:
  print('Reject H0 and Distribution is not Normal! ') 

In [ ]:
stats.probplot(df['Age'],plot=pylab,dist='norm')
pylab.show()

In [ ]:
df.head()

In [ ]:
df.groupby(df.Gender)['AnnualIncome'].describe()

In [ ]:
df.loc[df['Gender'] == 'Female','Gender'] = 0
df.loc[df['Gender'] == 'Male','Gender'] = 1

In [ ]:
df.head()

In [ ]:
kmeans = KMeans()
elbow = KElbowVisualizer(kmeans,k=(2,10))
elbow.fit(df)
elbow.poof()

In [ ]:
kmeans = KMeans(n_clusters=4).fit(df)

In [ ]:
result = pd.DataFrame({'Clusters':kmeans.labels_})

In [ ]:
kmeans.labels_

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = sns.countplot(data=result,x='Clusters')

In [ ]:
resultend = pd.DataFrame({'CustomerID':df.CustomerID,'Clusters':kmeans.labels_+1})

In [ ]:
resultend.head(200).T

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = sns.displot(data=resultend,x='Clusters',kde=True,kind='hist')

In [ ]:
kmeans_end = df.drop(['CustomerID', 'Gender'], axis=1)

In [ ]:
kmeans = KMeans()
elbow = KElbowVisualizer(kmeans,k=(2,10))
elbow.fit(kmeans_end)
elbow.poof()

In [ ]:
kmeansend = KMeans(n_clusters=5).fit(kmeans_end)

In [ ]:
kmeans_end['Labels'] = kmeansend.labels_
plt.figure(figsize=(12, 8))
sns.scatterplot(kmeans_end['AnnualIncome'], kmeans_end['SpendingScore'], hue=kmeans_end['Labels'], 
                palette=sns.color_palette('hls', 5))
plt.title('KMeans with 5 Clusters')
plt.show()

In [ ]:
# Dendrogram 
dist = distance_matrix(kmeans_end, kmeans_end)
dendrogram1 = hierarchy.linkage(dist, 'complete')
plt.figure(figsize=(18, 50))
dendrogram2 = hierarchy.dendrogram(dendrogram1, leaf_rotation=0, leaf_font_size=12, orientation='right')

***Thanks for looking. Please vote.***